<a href="https://colab.research.google.com/github/iued-uni-heidelberg/corpusdev/blob/main/part2_word_embeddings_v06_HIS_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word Vectors

Presentations [slides](intro-word-vectors-DAAD-2021-v22.pdf); Original [slides](word-vectors.pdf)

Word vectors (also known as 'word embeddings') are one of the most popular kinds of AI models. They are extremely useful in many domains. In essence, a word vector is a set of numbers that attempt to capture the meaning of a word. In typical implementations, each word is represented by a set of 200-300 numbers. In linear algebra, a one-dimensional array of numbers is known as a 'vector', hence these sets of numbers representing words' meanings are known as 'word vectors'.

Using neural networks, we can expose the computer to a large amount of text, and allow it to learn an appropriate set of numbers for each word it encounters. In this notebook, we will learn about the most famous of all word vector algorithms, `word2vec`, which was first described by Tomas Mikolov and his team in 2013:

* Tomas Mikolov, Ilya Sutskever, and others, ‘Distributed Representations of Words and Phrases and Their Compositionality’, in Advances in Neural Information Processing Systems 26, ed. by C. J. C. Burges and others (Curran Associates, Inc., 2013), pp. 3111–19 <http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf>
* Tomas Mikolov, Kai Chen, and others, ‘Efficient Estimation of Word Representations in Vector Space’, ArXiv:1301.3781 Cs, 2013 <http://arxiv.org/abs/1301.3781>.

In fact, `word2vec` is not a single algorithm, but rather a family of similar algorithms. In this session we will consider just the most famous `word2vec` algorithm, namely the `skip-gram model` trained using `negative sampling`.

## Applications of Word Vectors

Word vectors allow the computer to 'understand' language far more effectively. Rather than seeing each word as simply an arbitrarily different object, a computer using word vectors can analyse each word as a point in 200- or 300-dimenstional space. Words that are similar in meaning will have similar word vectors. And as we will see, the spaces between the word vectors are also significant: the words are arranged in patterns that represent their relationships to one another.

Accordingly, most AI systems that process language now include a word vector layer as part of their architecure. When the system encounters some text (e.g. when you speak to Siri or Alexa), your words are converted into word vectors, *and then* the computer examines what the text says and determines how it should respond.

In the Humanities, word vectors have become a popular modelling tool, because they allow researchers to perform sophisticated analysis on large corpora of text. Some examples include:

* [The Women Writers Vector Toolkit](https://wwp.northeastern.edu/lab/wwvt/index.html)
* William L. Hamilton, Jure Leskovec, and Dan Jurafsky, ‘Diachronic Word Embeddings Reveal Statistical Laws of Semantic Change’, ArXiv:1605.09096 [Cs], 2018 <http://arxiv.org/abs/1605.09096>.
* Ryan Heuser, 'Semantic Networks' <https://ryanheuser.org/word-vectors-4/>

## Training a `word2vec` model in Gensim


It is very easy to train a `word2vec` model in Gensim, which includes Mikolov's original `word2vec` code in its codebase.

### Step 0: Downloading texts
#### 0.1: load an existing text collection...


In [1]:
from gensim.models import Word2Vec # The word2vec model class
import gensim.downloader as api # Allows us to download some free training data
corpus = api.load('text8')
# api.info()

[==================================================] 100.0% 31.6/31.6MB downloaded


text8 is first 100MB from English wikipedia, 17 million words

The file is downloadable from
http://mattmahoney.net/dc/text8.zip


In [2]:
# Examine the corpus to see what is there
api.info("text8")

{'num_records': 1701,
 'record_format': 'list of str (tokens)',
 'file_size': 33182058,
 'reader_code': 'https://github.com/RaRe-Technologies/gensim-data/releases/download/text8/__init__.py',
 'license': 'not found',
 'description': 'First 100,000,000 bytes of plain text from Wikipedia. Used for testing purposes; see wiki-english-* for proper full Wikipedia datasets.',
 'checksum': '68799af40b6bda07dfa47a32612e5364',
 'file_name': 'text8.gz',
 'read_more': ['http://mattmahoney.net/dc/textdata.html'],
 'parts': 1}

In [3]:
type(corpus)

text8.Dataset

In [4]:
data = [d for d in corpus]

In [5]:
len(data)

1701

In [6]:
len(data[0])

10000

In [7]:
len(data[1])

10000

In [8]:
len(data[2])

10000

In [9]:
print(data[0][:100])

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst', 'the', 'term', 'is', 'still', 'used', 'in', 'a', 'pejorative', 'way', 'to', 'describe', 'any', 'act', 'that', 'used', 'violent', 'means', 'to', 'destroy', 'the', 'organization', 'of', 'society', 'it', 'has', 'also', 'been', 'taken', 'up', 'as', 'a', 'positive', 'label', 'by', 'self', 'defined', 'anarchists', 'the', 'word', 'anarchism', 'is', 'derived', 'from', 'the', 'greek', 'without', 'archons', 'ruler', 'chief', 'king', 'anarchism', 'as', 'a', 'political', 'philosophy', 'is', 'the', 'belief', 'that', 'rulers', 'are', 'unnecessary', 'and', 'should', 'be', 'abolished', 'although', 'there', 'are', 'differing']


In [10]:
print(data[1][:100])

['reciprocity', 'qualitative', 'impairments', 'in', 'communication', 'as', 'manifested', 'by', 'at', 'least', 'one', 'of', 'the', 'following', 'delay', 'in', 'or', 'total', 'lack', 'of', 'the', 'development', 'of', 'spoken', 'language', 'not', 'accompanied', 'by', 'an', 'attempt', 'to', 'compensate', 'through', 'alternative', 'modes', 'of', 'communication', 'such', 'as', 'gesture', 'or', 'mime', 'in', 'individuals', 'with', 'adequate', 'speech', 'marked', 'impairment', 'in', 'the', 'ability', 'to', 'initiate', 'or', 'sustain', 'a', 'conversation', 'with', 'others', 'stereotyped', 'and', 'repetitive', 'use', 'of', 'language', 'or', 'idiosyncratic', 'language', 'lack', 'of', 'varied', 'spontaneous', 'make', 'believe', 'play', 'or', 'social', 'imitative', 'play', 'appropriate', 'to', 'developmental', 'level', 'restricted', 'repetitive', 'and', 'stereotyped', 'patterns', 'of', 'behavior', 'interests', 'and', 'activities', 'as', 'manifested', 'by', 'at', 'least', 'one']


In [11]:
print(data[2][:100])

['with', 'the', 'aegis', 'of', 'zeus', 'when', 'he', 'goes', 'to', 'the', 'battlefield', 'the', 'entire', 'trojan', 'army', 'flees', 'behind', 'the', 'walls', 'of', 'troy', 'achilles', 'wrath', 'is', 'terrible', 'and', 'he', 'slays', 'many', 'trojan', 'warriors', 'and', 'allies', 'including', 'priam', 's', 'son', 'lycaon', 'whom', 'achilles', 'had', 'previously', 'captured', 'and', 'sold', 'into', 'slavery', 'but', 'who', 'had', 'been', 'returned', 'to', 'troy', 'eventually', 'hector', 'comes', 'out', 'of', 'the', 'walls', 'to', 'defend', 'the', 'honour', 'of', 'troy', 'he', 'asked', 'achilles', 'to', 'agree', 'that', 'the', 'body', 'of', 'the', 'loser', 'would', 'be', 'returned', 'for', 'proper', 'burial', 'by', 'the', 'winner', 'achilles', 'rejected', 'this', 'arrangement', 'saying', 'though', 'twenty', 'ransoms', 'and', 'thy', 'weight', 'in', 'gold']


#### Step 0.2: load own text collection to be used in training ... (will be added later)...


### Step 1: Set hyperparameters and instantiate model

In [12]:
my_vector_size = 100 # Dimensionality of the word vectors
window = 5 # How many words either side? (5 = 5 context words either side, i.e. 10 context words in total)
use_skip_gram = 1 # If you set this to 0, then it will create a 'continuous bag of words' model instead
use_softmax = 0 # If you set this to 1, then hierarchical softmax will be used instead of negative sampling
negative_samples = 5 # How many incorrect answers to generate per correct answer when negative sampling

model = Word2Vec(
    vector_size=my_vector_size,
    window=window,
    sg=use_skip_gram,
    hs=use_softmax,
    negative=negative_samples
)

### Step 2: Fit model to corpus

In [13]:
# build a model
model.build_vocab(corpus)

In [11]:
# Train the model on the corpus
# model.train(sentences=corpus, epochs=5, total_examples=model.corpus_count)
# runs approximately 10 minutes with 5 epochs...
model.train(corpus, epochs=5, total_examples=model.corpus_count)

# saving the model, zip can be downloaded...
model.save("word2vec5e.model")
!zip word2vec5e.model.zip word2vec5e.model
!rm word2vec5e.model

(62529375, 85026035)

In [ ]:
# modelOwn.train(sentences=corpusOwn, epochs=5, total_examples=modelOwn.corpus_count)

In [41]:
## !rm word2vec.model

### Step 2: alternative: downloading a pre-trained model

In [14]:
## Downloading and connecting pre-trained model
!wget https://heibox.uni-heidelberg.de/f/b7625cd746534aacbefe/?dl=1
!mv index.html?dl=1 word2vec.model

--2024-06-09 14:38:38--  https://heibox.uni-heidelberg.de/f/b7625cd746534aacbefe/?dl=1
Resolving heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)... 129.206.7.113
Connecting to heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)|129.206.7.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://heibox.uni-heidelberg.de/seafhttp/files/2dc0b8fa-7013-495a-bf24-9bd188c1ce72/word2vec.model [following]
--2024-06-09 14:38:38--  https://heibox.uni-heidelberg.de/seafhttp/files/2dc0b8fa-7013-495a-bf24-9bd188c1ce72/word2vec.model
Reusing existing connection to heibox.uni-heidelberg.de:443.
HTTP request sent, awaiting response... 200 OK
Length: 59217897 (56M) [application/octet-stream]
Saving to: ‘index.html?dl=1’

index.html?dl=1     100%[===================>]  56.47M  17.0MB/s    in 3.5s    

2024-06-09 14:38:42 (16.2 MB/s) - ‘index.html?dl=1’ saved [59217897/59217897]



In [15]:
model = Word2Vec.load("word2vec.model")
# model2 = Word2Vec.load("word2vec.model")

### Step 3: Extract word vectors from model

The fully trained model includes all of the weights used to predict the context words for each input word. If you are not planning on training the model further, these weights can be discarded, and you can just keep the weights for the word vectors.

In [16]:
word_vectors = model.wv
del model # Delete the whole model to free up the computer's RAM

In [17]:
word_vectors.save("word2vec.wordvectors5e")
!zip word2vec.wordvectors5e.zip word2vec.wordvectors5e
!rm word2vec.wordvectors5e

  adding: word2vec.wordvectors5e (deflated 12%)


In [42]:
## !rm word2vec.wordvectors

### Step 3 alternative: download and connect the pre-trained word vectors

In [21]:
!wget https://heibox.uni-heidelberg.de/f/e7725b1afab9456c8e16/?dl=1
!mv index.html?dl=1 word2vec.wordvectors

--2024-06-09 14:27:03--  https://heibox.uni-heidelberg.de/f/e7725b1afab9456c8e16/?dl=1
Resolving heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)... 129.206.7.113
Connecting to heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)|129.206.7.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://heibox.uni-heidelberg.de/seafhttp/files/987d80a8-aee6-4f3f-b788-e95bb1a6fe27/word2vec.wordvectors [following]
--2024-06-09 14:27:04--  https://heibox.uni-heidelberg.de/seafhttp/files/987d80a8-aee6-4f3f-b788-e95bb1a6fe27/word2vec.wordvectors
Reusing existing connection to heibox.uni-heidelberg.de:443.
HTTP request sent, awaiting response... 200 OK
Length: 30697035 (29M) [application/octet-stream]
Saving to: ‘index.html?dl=1’

index.html?dl=1     100%[===================>]  29.27M  15.8MB/s    in 1.9s    

2024-06-09 14:27:06 (15.8 MB/s) - ‘index.html?dl=1’ saved [30697035/30697035]



In [22]:
from gensim.models import KeyedVectors
word_vectors = KeyedVectors.load("word2vec.wordvectors", mmap='r')

### Step 4: exploring word vectors

In [18]:
vector = word_vectors['computer']

In [19]:
vector

array([ 0.06077942,  0.1819358 , -0.05054147,  0.12852807,  0.01770034,
       -0.30997637, -0.24055079, -0.00272755, -0.6581456 , -0.18392453,
        0.06989374, -0.5224778 , -0.20864458, -0.10262147, -0.22059658,
        0.33618814,  0.04703446, -0.47088787, -0.1487342 , -0.6229217 ,
        0.42406246, -0.5138749 ,  0.06158032, -0.63825554, -0.07985036,
        0.16819875,  0.09989165, -0.17658606, -0.17862448,  0.29675245,
        0.4758706 ,  0.48978546,  0.9608675 , -0.22739977,  0.19066736,
        0.08846364,  0.07009576, -0.360089  , -0.06736434, -0.21808966,
       -0.2349182 , -0.25443017,  0.53662956,  0.2744204 ,  0.00105796,
       -0.12268312, -0.08814082,  0.00657543,  0.48798513, -0.11536589,
        0.2614905 ,  0.51717365, -0.04133837, -0.3994545 , -0.654159  ,
       -0.40764585,  0.7202135 , -0.2568953 ,  0.52592474,  0.11868921,
        0.1729699 ,  0.5800076 ,  0.46083173,  0.28301194,  0.58508384,
        0.46322995,  0.34925246,  0.02775881, -0.01378645,  0.07

### Step 4: Have a play with the model

There are several ways you can use word vectors. One of the most famous is to use them to compute analogies. The formula is:

<center><em>x</em> is to <em>small</em> as <em>biggest</em> is to <em>big</em></center>

$$x - vector('small') = vector('biggest') - vector('big')$$

$$\therefore x = vector('small') + vector('biggest') - vector('big')$$

In [20]:
# See the word vector for a particular word
vector = word_vectors['banana']
print(vector)

[ 0.04514017  0.3785197  -0.45157424 -0.53498334 -0.3810624  -0.19748794
  0.3861872  -0.09874219 -0.3083414  -0.13589764 -0.19032787 -0.186255
 -0.24850576  0.15236585 -0.18161802 -0.28916776  0.26343766 -0.22076558
 -0.32832116 -0.18427545  0.16742764  0.13754863  0.8236313   0.1276648
 -0.51062804  0.04845584 -0.2820541  -0.32418352  0.31521824  0.34778962
 -0.09972392  0.21742499  0.3331799   0.16179839  0.06564191 -0.16745356
 -0.26856118  0.20320536 -0.218172   -0.8700633   0.28307882 -0.11479696
  0.2597842  -0.14202665  0.19578132 -0.08296769 -0.5186016  -0.07467933
  0.54540306  0.19925955  0.13505691 -0.34502357  0.54741985  0.19840081
 -0.35757104  0.30361545  0.5539709   0.41801247 -0.3125661   0.28622827
  0.16626568  0.30216348 -0.23019     0.10558449 -0.0202124  -0.19527975
 -0.0798233   0.35605833 -0.0533659  -0.40334567 -0.19850768  0.6096959
 -0.00360445  0.10260782  0.42559785  0.29462764 -0.45577955  0.14467828
 -0.11372449 -0.01252147  0.28607625 -0.49714658 -0.130

In [21]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors.most_similar('university', topn=25)
print('\n'.join([str(tup) for tup in similar_words]))

('college', 0.8274326920509338)
('urbana', 0.8046768307685852)
('champaign', 0.8018592000007629)
('polytechnic', 0.778497576713562)
('rutgers', 0.7744151949882507)
('institute', 0.7612160444259644)
('emory', 0.7561977505683899)
('abet', 0.7453380227088928)
('tsinghua', 0.7393555641174316)
('graduate', 0.7371058464050293)
('pomona', 0.7350606918334961)
('mellon', 0.7330489754676819)
('polytechnical', 0.7320426106452942)
('fordham', 0.7311391234397888)
('witwatersrand', 0.7299140095710754)
('abertay', 0.7293667793273926)
('phd', 0.7263227701187134)
('emeritus', 0.7218440771102905)
('cambridge', 0.7216722965240479)
('professor', 0.7202757596969604)
('rensselaer', 0.7194692492485046)
('conservatory', 0.7186447381973267)
('amherst', 0.7155526876449585)
('vassar', 0.7155349254608154)
('ryerson', 0.7145218253135681)


In [22]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors.most_similar('red', topn=25)
print('\n'.join([str(tup) for tup in similar_words]))

('blue', 0.8330671787261963)
('yellow', 0.8198412656784058)
('green', 0.7955173850059509)
('white', 0.7705235481262207)
('purple', 0.7580312490463257)
('black', 0.7194028496742249)
('magenta', 0.696144700050354)
('orange', 0.6725648641586304)
('stripes', 0.6695405840873718)
('sash', 0.668810248374939)
('brindle', 0.6663251519203186)
('stockings', 0.6570967435836792)
('stripe', 0.6544612646102905)
('jerseys', 0.6513431072235107)
('colored', 0.6477282047271729)
('talons', 0.6458464860916138)
('hoist', 0.6443328261375427)
('coloured', 0.6427997350692749)
('crescent', 0.6411023736000061)
('facemask', 0.6405667662620544)
('striped', 0.6376775503158569)
('pink', 0.6366048455238342)
('brownish', 0.6365171670913696)
('dwarfs', 0.6351383328437805)
('colors', 0.6344648003578186)


In [23]:
similar_words = word_vectors.most_similar('color', topn=25)
print('\n'.join([str(tup) for tup in similar_words]))

('colour', 0.8506642580032349)
('colors', 0.8253850936889648)
('cyan', 0.7517661452293396)
('palettes', 0.7500985860824585)
('rgb', 0.732881486415863)
('colored', 0.7236745953559875)
('monochrome', 0.7235877513885498)
('colours', 0.7123246788978577)
('yellow', 0.7095232605934143)
('srgb', 0.697119414806366)
('cmyk', 0.6920041441917419)
('blue', 0.6906263828277588)
('chroma', 0.6890067458152771)
('pixel', 0.6886876225471497)
('hue', 0.6886435151100159)
('shade', 0.676449179649353)
('subpixels', 0.6758597493171692)
('hues', 0.6755530834197998)
('pantone', 0.6750978231430054)
('coloration', 0.6709157228469849)
('palette', 0.6701295971870422)
('monochromatic', 0.6698700189590454)
('dddddd', 0.6678876876831055)
('pixels', 0.6621844172477722)
('gif', 0.6574938893318176)


In [24]:
similar_words = word_vectors.most_similar('colour', topn=25)
print('\n'.join([str(tup) for tup in similar_words]))

('color', 0.8506643176078796)
('colored', 0.7899959087371826)
('colours', 0.7802109718322754)
('colors', 0.7580424547195435)
('cyan', 0.7343934774398804)
('palettes', 0.7335439324378967)
('coloration', 0.7164565324783325)
('shade', 0.7106713652610779)
('hues', 0.7073876857757568)
('shades', 0.704552948474884)
('monochrome', 0.6996116042137146)
('palette', 0.6918415427207947)
('lettering', 0.6916214823722839)
('glasses', 0.6900200247764587)
('coloured', 0.6897372603416443)
('tint', 0.6894671320915222)
('brownish', 0.6887184977531433)
('yellowish', 0.6851990222930908)
('texture', 0.6837971806526184)
('dddddd', 0.682521402835846)
('grayish', 0.6803927421569824)
('bluish', 0.6801412105560303)
('tinctures', 0.6787596344947815)
('pigment', 0.6772411465644836)
('monochromatic', 0.6757177114486694)


In [25]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors.most_similar('brussels', topn=25)
print('\n'.join([str(tup) for tup in similar_words]))

('aachen', 0.7496265172958374)
('ghent', 0.7475510835647583)
('strasbourg', 0.7462074160575867)
('paris', 0.7316218018531799)
('bruges', 0.7294041514396667)
('maastricht', 0.7258070111274719)
('commune', 0.7200930118560791)
('bordeaux', 0.7187761664390564)
('frankfurt', 0.7151591777801514)
('antwerp', 0.7144349813461304)
('ticino', 0.7100759148597717)
('rotterdam', 0.7072901129722595)
('hamburg', 0.7057906985282898)
('cologne', 0.7045364379882812)
('conservatoire', 0.7045349478721619)
('amsterdam', 0.7040742039680481)
('seine', 0.7038989067077637)
('bruxelles', 0.7029331922531128)
('lausanne', 0.6956555247306824)
('basel', 0.6928088068962097)
('rennes', 0.6912458539009094)
('barcelona', 0.6911035180091858)
('bonn', 0.6901640295982361)
('namur', 0.6892477869987488)
('dieppe', 0.6890778541564941)


In [26]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors.most_similar('heidelberg', topn=25)
print('\n'.join([str(tup) for tup in similar_words]))

('freiburg', 0.8832603693008423)
('halle', 0.8257067203521729)
('ingolstadt', 0.8146023750305176)
('ttingen', 0.8137246966362)
('linz', 0.7996790409088135)
('jena', 0.7928030490875244)
('bingen', 0.792185366153717)
('leiden', 0.7865665555000305)
('wien', 0.784911036491394)
('breslau', 0.7835977077484131)
('kassel', 0.7818834781646729)
('grammatik', 0.7785252332687378)
('fulda', 0.7702142596244812)
('regensburg', 0.7701405882835388)
('karlsruhe', 0.7684836387634277)
('laude', 0.764237642288208)
('sorbonne', 0.7637717723846436)
('bologna', 0.7631106376647949)
('professorship', 0.7615196108818054)
('strassburg', 0.7607799768447876)
('erfurt', 0.7604721188545227)
('graz', 0.7602874636650085)
('preis', 0.7584127187728882)
('marburg', 0.7572142481803894)
('wissenschaft', 0.7568274140357971)


In [27]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors.most_similar('international', topn=25)
print('\n'.join([str(tup) for tup in similar_words]))

('electrotechnical', 0.7461802959442139)
('hydrographic', 0.6888812184333801)
('ilo', 0.6752741932868958)
('radiocommunications', 0.6744605302810669)
('nongovernmental', 0.6731569766998291)
('bipm', 0.6643987894058228)
('organization', 0.6642566323280334)
('paralympic', 0.6551176905632019)
('workingmen', 0.6540185213088989)
('imo', 0.6540013551712036)
('undp', 0.6524684429168701)
('intergovernmental', 0.6477717161178589)
('meteorological', 0.6472776532173157)
('imd', 0.6462644934654236)
('medarabtel', 0.6447435617446899)
('supranationalism', 0.6445252895355225)
('icsu', 0.6422662138938904)
('blissymbolics', 0.6421657204627991)
('acct', 0.6416999101638794)
('abeda', 0.6402809619903564)
('mesures', 0.6396852731704712)
('inmarsat', 0.6385820508003235)
('gatt', 0.6379938721656799)
('lome', 0.6366467475891113)
('asdb', 0.6352181434631348)


In [28]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors.most_similar('germany', topn=25)
print('\n'.join([str(tup) for tup in similar_words]))

('austria', 0.8406285643577576)
('hungary', 0.7882422804832458)
('italy', 0.7679662108421326)
('denmark', 0.7570524215698242)
('berlin', 0.7491813898086548)
('poland', 0.7450143694877625)
('munich', 0.7411389946937561)
('belgium', 0.7378813624382019)
('czechoslovakia', 0.7369067668914795)
('bavaria', 0.7355030179023743)
('russia', 0.7344875335693359)
('nazi', 0.7332985997200012)
('finland', 0.7295423746109009)
('switzerland', 0.7270355224609375)
('prussia', 0.724307656288147)
('hitler', 0.7138442993164062)
('romania', 0.7135593295097351)
('weimar', 0.7083632349967957)
('rhineland', 0.7074394226074219)
('ukraine', 0.7045464515686035)
('hamburg', 0.7030502557754517)
('sudetenland', 0.7023268938064575)
('alsace', 0.7022143006324768)
('europe', 0.7017481327056885)
('silesia', 0.7015114426612854)


In [29]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors.most_similar('strange', topn=25)
print('\n'.join([str(tup) for tup in similar_words]))

('mysterious', 0.6891576051712036)
('hilarious', 0.6680781841278076)
('darkened', 0.6523415446281433)
('improbable', 0.6510730385780334)
('startling', 0.6504903435707092)
('dread', 0.6504672765731812)
('kaleidoscope', 0.6462839245796204)
('crazy', 0.6390725374221802)
('monstrous', 0.6363013982772827)
('marvellous', 0.6358059644699097)
('martian', 0.6357612013816833)
('stranger', 0.6328702569007874)
('androids', 0.6318789720535278)
('fantastic', 0.6301376819610596)
('marvelous', 0.6299870014190674)
('tenderness', 0.6270742416381836)
('comical', 0.6268086433410645)
('lonely', 0.6265559792518616)
('surreal', 0.6265548467636108)
('nightmarish', 0.6227016448974609)
('torrent', 0.6225694417953491)
('rogues', 0.6221829056739807)
('amazing', 0.6219238638877869)
('cheerful', 0.6219094395637512)
('hideous', 0.6218962669372559)


In [30]:
# Compute analogous words
# E.g. x is to queen as man is to king => x = v('queen') + v('man') - v('king')
analogous_words = word_vectors.most_similar(negative=['man'], positive=['king','woman'], topn=20)
print('\n'.join([str(tup) for tup in analogous_words]))

('queen', 0.6899670958518982)
('daughter', 0.6563656330108643)
('throne', 0.6470769643783569)
('jadwiga', 0.6346895098686218)
('matilda', 0.6335664391517639)
('princess', 0.6311494708061218)
('empress', 0.6252461671829224)
('consort', 0.6083589196205139)
('alexandra', 0.6075658798217773)
('prince', 0.6042875647544861)
('wenceslaus', 0.6036732196807861)
('dowager', 0.60321444272995)
('marries', 0.5995404124259949)
('kammu', 0.5958123803138733)
('sigismund', 0.5940837860107422)
('son', 0.5914080142974854)
('anjou', 0.5864495038986206)
('pretender', 0.5860036015510559)
('jeroboam', 0.5842873454093933)
('heir', 0.5839943885803223)


In [31]:
# Compute analogous words
# E.g. x is to queen as man is to king => x = v('queen') + v('man') - v('king')
analogous_words = word_vectors.most_similar(negative=['king'], positive=['queen','man'])
print('\n'.join([str(tup) for tup in analogous_words]))

('woman', 0.7189967036247253)
('blonde', 0.6277213096618652)
('girl', 0.6162012815475464)
('lady', 0.6079869270324707)
('frail', 0.5973320007324219)
('gentleman', 0.5871325731277466)
('prostitute', 0.5832652449607849)
('pauper', 0.5768119692802429)
('cinderella', 0.5749097466468811)
('doll', 0.5737509727478027)


In [32]:
# Compute analogous words
# E.g. x is to queen as man is to king => x = v('queen') + v('man') - v('king')
analogous_words = word_vectors.most_similar(negative=['mother'], positive=['father','daughter'])
print('\n'.join([str(tup) for tup in analogous_words]))

('son', 0.8429544568061829)
('brother', 0.7873624563217163)
('nephew', 0.7757642865180969)
('wife', 0.7564918994903564)
('eldest', 0.7470341920852661)
('grandson', 0.7452330589294434)
('cousin', 0.7400377988815308)
('granddaughter', 0.7250849008560181)
('uncle', 0.720912516117096)
('confidant', 0.7121458649635315)


In [33]:
# Compute analogous words
# E.g. x is to queen as man is to king => x = v('queen') + v('man') - v('king')
analogous_words = word_vectors.most_similar(negative=['mother'], positive=['father','girl'])
print('\n'.join([str(tup) for tup in analogous_words]))

('bartender', 0.6588817834854126)
('boy', 0.6539564728736877)
('goodnight', 0.6461302042007446)
('lebowski', 0.6418622136116028)
('man', 0.6416395306587219)
('tights', 0.6400418877601624)
('naughty', 0.6371098756790161)
('guys', 0.6361229419708252)
('cinderella', 0.6335315108299255)
('bogart', 0.6333711743354797)


In [34]:
# Compute analogous words
# x is to daughter as people is to person (plural + daughter)
analogous_words = word_vectors.most_similar(negative=['person'], positive=['people','daughter'])
print('\n'.join([str(tup) for tup in analogous_words]))

('daughters', 0.6566089391708374)
('sons', 0.6294084787368774)
('alexandra', 0.6059257388114929)
('henrietta', 0.5997800230979919)
('elisabeth', 0.5861902832984924)
('brothers', 0.5859265923500061)
('offa', 0.5770557522773743)
('archduchess', 0.574726402759552)
('eleonora', 0.574079155921936)
('teschen', 0.5718636512756348)


In [35]:
analogous_words = word_vectors.most_similar(negative=['soft'], positive=['hard','softer'])
# analogous_words = word_vectors.most_similar(negative=['soft'], positive=['hard','softest'])
print('\n'.join([str(tup) for tup in analogous_words]))

('harder', 0.708805501461029)
('louder', 0.6695201992988586)
('smoother', 0.6673597097396851)
('rougher', 0.6625217199325562)
('easier', 0.6569008827209473)
('simpler', 0.6523188352584839)
('thicker', 0.6513201594352722)
('consonances', 0.645392894744873)
('quicker', 0.6449112892150879)
('slower', 0.6432040929794312)


## Step 4A - optional. Using pre-trained models in Gensim

In many applications, you will simply want access to pre-trained word vectors (e.g. for plugging in to another model you are training). If you don't need the vectors to be tailored closely to your particular corpus, then you might like to use some pretrained models.

`word2vec` is not the only word embedding family of algorithms. Another, arguably even more powerful algorithm is the `FastText` algorithm, which Mikolov developed after moving to Facebook:

* Piotr Bojanowski and others, ‘Enriching Word Vectors with Subword Information’, ArXiv:1607.04606, 2017 <http://arxiv.org/abs/1607.04606>.

Instead of computing word vectors for each word, FastText splits each word into its constituent chunks. For example, 'cat' would be split into 'c', 'a', 't', 'ca', 'at' and 'cat', and 'burp' would be split into 'b', 'u', 'r', 'p', 'bu', 'ur', 'rp', 'bur', 'urp' and 'burp'. Then a vector is computer for each chunk that appears in the corpus. Each word is represented as the mean of all the chunks that make it up. FastText is able to learn very good word vectors because it can extract meaning from subword units, e.g. it can see that 'television', 'telegraph' and 'telepathy' all have 'tele' at the front, and can see that 'formality', 'criminality' and 'paucity' share subword units such as 'al' and 'ity'.

You can access many pretrained models using the Gensim downloader. Using the cells below, you can try out some of the different models available through Gensim. Along with `word2vec` and `FastText`, Gensim also supports `Glove` and `Doc2Vec` models.

**NB:** These trained models are very large, and will take a while to download. You may wish to download this notebook and execute the cells below on your own machine, in case Google kicks you out of the Colab environment.

In [36]:
# See what models are on offer
print(list(api.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


#### word2vec-google-news-300

In [38]:
# Optional! THIS WILL TAKE SOME TIME!!! (~15 min)
# 300-dimensional word vectors trained on a huge dataset from Google News
google_news_w2v = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [42]:
analogous_words = google_news_w2v.most_similar(negative=['soft'], positive=['hard','softer'])
print('\n'.join([str(tup) for tup in analogous_words]))

('harder', 0.713630735874176)
('tougher', 0.5499427914619446)
('tough', 0.5147612690925598)
('better', 0.5042758584022522)
('difficult', 0.49098414182662964)
('stronger', 0.4880489110946655)
('Hard', 0.48069724440574646)
('easier', 0.46995335817337036)
('hardest', 0.4648631513118744)
('lot', 0.4465230405330658)


In [45]:
# x is to Kenya as Canberra is to Australia
google_news_w2v.most_similar(negative=['australia'],positive=['kenya','canberra'], topn=10)

[('kampala', 0.5206839442253113),
 ('na_wa', 0.5155055522918701),
 ('nairobi', 0.5032686591148376),
 ('JZU', 0.5002182126045227),
 ('9ja', 0.5001271963119507),
 ('RAZAK', 0.49960625171661377),
 ('ngoba', 0.49234238266944885),
 ('abuja', 0.49174243211746216),
 ('chete', 0.48337095975875854),
 ('naija', 0.48265454173088074)]

#### glove_wiki_gigaword_300

In [40]:
# download time ~ 4 min
glove_wiki_gigaword_300 = api.load('glove-wiki-gigaword-300')

[==================================================] 100.0% 376.1/376.1MB downloaded


In [46]:
analogous_words = glove_wiki_gigaword_300.most_similar(negative=['soft'], positive=['hard','softer'])
print('\n'.join([str(tup) for tup in analogous_words]))

('harder', 0.6572688817977905)
('stronger', 0.5350075960159302)
('tougher', 0.5323050618171692)
('tough', 0.5159251093864441)
('better', 0.5049951672554016)
('easier', 0.47591423988342285)
('lot', 0.4734606444835663)
('much', 0.46252939105033875)
('weaker', 0.46007463335990906)
('bit', 0.4486900568008423)


#### wikipedia_fasttext

In [43]:
# Optional! THIS WILL TAKE SOME TIME!!! (download time ~10 minutes)
# Facebook's own FastText vectors, trained on Wikipedia
wikipedia_fasttext = api.load('fasttext-wiki-news-subwords-300')

[==================================================] 100.0% 958.5/958.4MB downloaded


In [47]:
analogous_words = wikipedia_fasttext.most_similar(negative=['soft'], positive=['hard','softer'])
print('\n'.join([str(tup) for tup in analogous_words]))

('harder', 0.8653953671455383)
('tougher', 0.7348615527153015)
('easier', 0.6936573386192322)
('better', 0.6659234166145325)
('tough', 0.6521403789520264)
('hardest', 0.6454681158065796)
('difficult', 0.6439278721809387)
('rougher', 0.6428141593933105)
('harsher', 0.6367827653884888)
('stronger', 0.6283641457557678)


In [55]:
# x is to Wharton as London is to Dickens
analogous_words = wikipedia_fasttext.most_similar(negative=['paris'],positive=['france','london'], topn=10)
print('\n'.join([str(tup) for tup in analogous_words]))

('england', 0.6565924882888794)
('britain', 0.628123939037323)
('europe', 0.6154525876045227)
('london.', 0.5932062864303589)
('londons', 0.5780512690544128)
('uk', 0.5756634473800659)
('british', 0.5626298189163208)
('scotland', 0.5594385266304016)
('lond', 0.5568357110023499)
('germany', 0.5533894896507263)


In [ ]:
analogous_words = glove_wiki_gigaword_300.most_similar(negative=['soft'], positive=['hard','softer'])
print('\n'.join([str(tup) for tup in analogous_words]))

In [ ]:
# try your own examples...

## Step 5. Training models in other languages (Armenian, Georgian, Ukrainian ...)

Now let's train a model on our own corpus. You can try to create Armenian / Ukrainian / Russian / German / French word vector model.

Armenian, Ukrainian, Russian plaintext wikipedias are available at:
https://lindat.mff.cuni.cz/repository/xmlui/handle/11234/1-2735#

We can use the same size ~15 Million Words, (200MB) of these corpora to make sure the model is built within a reasonable time; however, when you have more time, you can try a more complete corpus.

It is possible to download/unzip/upload the files manually, however, it is faster to automatically load them directly from Heidelberg server space https://heibox.uni-heidelberg.de/d/42a07c9e95774e099a81/  into your colab environment. To do this run the cells 5a or 5b, or 5c, or 5d, or 5e -- depending on which language you prefer -- to download the data. Then skip other language and continue with Step5 -- building the model for your preferred language from the data you downloaded.


In [ ]:
!rm /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt
!rm index.html?dl=1

rm: cannot remove '/usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt': No such file or directory
rm: cannot remove 'index.html?dl=1': No such file or directory


### 5c German - works!

In [62]:
!wget https://heibox.uni-heidelberg.de/f/ca0a085347c34563a6e6/?dl=1

--2024-06-09 16:40:52--  https://heibox.uni-heidelberg.de/f/ca0a085347c34563a6e6/?dl=1
Resolving heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)... 129.206.7.113
Connecting to heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)|129.206.7.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://heibox.uni-heidelberg.de/seafhttp/files/0040f62a-ead1-42a2-9dff-1f1edfd7debf/de8.txt [following]
--2024-06-09 16:40:52--  https://heibox.uni-heidelberg.de/seafhttp/files/0040f62a-ead1-42a2-9dff-1f1edfd7debf/de8.txt
Reusing existing connection to heibox.uni-heidelberg.de:443.
HTTP request sent, awaiting response... 200 OK
Length: 120000000 (114M) [text/plain]
Saving to: ‘index.html?dl=1’

index.html?dl=1     100%[===================>] 114.44M  14.9MB/s    in 7.3s    

2024-06-09 16:41:00 (15.6 MB/s) - ‘index.html?dl=1’ saved [120000000/120000000]



In [63]:
!cp index.html?dl=1 /usr/local/lib/python3.10/dist-packages/gensim/test/test_data/myOwnLangText8.txt

In [64]:
!head --lines=15 /usr/local/lib/python3.10/dist-packages/gensim/test/test_data/myOwnLangText8.txt

Alan Smithee

Alan Smithee steht als Pseudonym für einen fiktiven Regisseur, der Filme verantwortet, bei denen der eigentliche Regisseur seinen Namen nicht mit dem Werk in Verbindung gebracht haben möchte. Von 1968 bis 2000 wurde es von der Directors Guild of America (DGA) für solche Situationen empfohlen, seither ist es Thomas Lee. "Alan Smithee" ist jedoch weiterhin in Gebrauch.

Alternative Schreibweisen sind unter anderem die Ursprungsvariante "Allen Smithee" sowie "Alan Smythee" und "Adam Smithee". Auch zwei teilweise asiatisch anmutende Schreibweisen "Alan Smi Thee" und "Sumishii Aran" gehören – so die Internet Movie Database – dazu.

Das Pseudonym entstand 1968 infolge der Arbeiten am Western-Film "Death of a Gunfighter" (deutscher Titel "Frank Patch – Deine Stunden sind gezählt"). Regisseur Robert Totten und Hauptdarsteller Richard Widmark gerieten in einen Streit, woraufhin Don Siegel als neuer Regisseur eingesetzt wurde.

Der Film trug nach Abschluss der Arbeiten noch deutlic

### Or: 5a Armenian

In [ ]:
!wget https://heibox.uni-heidelberg.de/f/ffe527ed3d1e4b4cb43a/?dl=1

In [ ]:
!cp index.html?dl=1 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

In [ ]:
!head --lines=15 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

### Or: 5b Ukrainian

In [ ]:
!wget https://heibox.uni-heidelberg.de/f/676bef3a6db8482e9665/?dl=1

In [ ]:
!cp index.html?dl=1 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

In [ ]:
!head --lines=15 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

### Or: 5d French

In [ ]:
!wget https://heibox.uni-heidelberg.de/f/5cdb485efd4046f2a457/?dl=1

In [ ]:
!cp index.html?dl=1 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

In [ ]:
!head --lines=15 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

### Or: 5f Georgian

In [ ]:
!wget https://heibox.uni-heidelberg.de/f/fa3509d869b949459f91/?dl=1

Lemmatized corpus

In [ ]:
!wget https://heibox.uni-heidelberg.de/f/b381c458ad5e4773a77d/?dl=1

In [ ]:
!cp index.html?dl=1 wiki_geo_lem.txt

In [ ]:
!head --lines=10 wiki_geo_lem.txt

In [ ]:
FIn = open("wiki_geo_lem.txt", 'r')
FOut = open('wiki_geo_l.txt', 'w')

i = 0
for SLine in FIn:
  i = i + 1
  SLine = SLine.strip() + ' '
  if i % 10000 == 0:
    SLine += '\n'

  FOut.write(SLine)

FOut.flush()
FOut.close()


In [ ]:
!head --lines=10 wiki_geo_l.txt

In [ ]:
!wc wiki_geo_lem.txt

 18054750  18054749 329202500 wiki_geo_lem.txt


In [ ]:
!cp index.html?dl=1 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

Alternatively, copy lemmatized corpus

In [ ]:
!cp wiki_geo_l.txt /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

In [ ]:
!head --lines=15 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

### Stage6: Training own model

optional: to clean disk space, we remove the downloaded file

In [65]:
# optional: to clean disk space, we remove the downloaded file
!rm index.html\?dl\=1

In [66]:
from gensim.test.utils import datapath
from gensim import utils

class MyCorpus:
    """An iterator that yields sentences (lists of str)."""

    def __iter__(self):
        corpus_path = datapath('myOwnLangText8.txt')
        for line in open(corpus_path):
            # assume there's one document per line, tokens separated by whitespace
            yield utils.simple_preprocess(line)

In [67]:
import gensim.models
corpusOwn = MyCorpus()

optional: examining what is in the corpus after standard preprocessing

In [68]:
# Optional: Examining our corpus format
type(corpusOwn)

__main__.MyCorpus

In [69]:
dataOwn = [d for d in corpusOwn]

In [70]:
print(len(dataOwn))
print(dataOwn[0])
print(len(dataOwn[0]))
print(dataOwn[4])
print(len(dataOwn[4]))
print(dataOwn[5])
print(len(dataOwn[5]))

617267
['alan', 'smithee']
2
['alternative', 'schreibweisen', 'sind', 'unter', 'anderem', 'die', 'allen', 'smithee', 'sowie', 'alan', 'smythee', 'und', 'adam', 'smithee', 'auch', 'zwei', 'teilweise', 'asiatisch', 'anmutende', 'schreibweisen', 'alan', 'smi', 'thee', 'und', 'sumishii', 'aran', 'gehören', 'so', 'die', 'internet', 'movie', 'database', 'dazu']
33
[]
0


... Initialising global parameters for our modelL vector size, collocation window, skip-grams, negative sampling....

In [71]:
from gensim.models import Word2Vec # The word2vec model class
import gensim.downloader as api # Allows us to download some free training data

In [73]:
my_vector_size = 100 # Dimensionality of the word vectors
window = 5 # How many words either side? (5 = 5 context words either side, i.e. 10 context words in total)
use_skip_gram = 1 # If you set this to 0, then it will create a 'continuous bag of words' model instead
use_softmax = 0 # If you set this to 1, then hierarchical softmax will be used instead of negative sampling
negative_samples = 5 # How many incorrect answers to generate per correct answer when negative sampling

modelOwn = Word2Vec(
    vector_size=my_vector_size,
    window=window,
    sg=use_skip_gram,
    hs=use_softmax,
    negative=negative_samples
)

... this cell may run for ~2 min or so...

In [74]:
modelOwn.build_vocab(corpusOwn)

THIS MAY TAKE LONG!!! ... training the model may take 9 to 15 minutes... (just grab a cup of coffee or a sandwich while you are waiting... You may try chaning the number of epochs; if the number is lower the training is faster, but the quality may be lower...

In [76]:
modelOwn.train(corpusOwn, epochs=5, total_examples=modelOwn.corpus_count)

(56971850, 75507725)

Now we copy word vectors and remove the model from memory (just to free up the resources...)

In [77]:
word_vectors_own = modelOwn.wv

In [ ]:
del modelOwn

Now we can examine the output of our word embeddings

### German examples

In [78]:
# DE examples
# See the word vector for a particular word
vector = word_vectors_own['banane']
print(vector)

[ 0.10075621 -0.12602118 -0.09882525  0.07208958  0.0338595  -0.1839196
  0.279739    0.31493902 -0.07504811 -0.18543933 -0.14300443 -0.08045007
 -0.20416714  0.09101471  0.03044987 -0.22884238  0.05614935  0.05093222
  0.24569729 -0.24122925  0.06150239 -0.20853686  0.16781837 -0.15910992
 -0.24888885  0.14393839  0.058139   -0.07789802 -0.0904852  -0.10539671
  0.22656076 -0.22272146  0.1546064   0.17589416  0.01947307  0.2050054
 -0.08090865 -0.17477393 -0.22255301  0.10397669  0.17486668 -0.27339187
  0.2600213   0.16386129 -0.22488172 -0.14953832 -0.06997634 -0.17026299
 -0.0010755   0.19792816  0.3408265   0.05118159 -0.33898032 -0.03472055
 -0.29269937  0.03965892  0.04215126  0.19848008 -0.06039856 -0.1654338
 -0.03749323 -0.09299678  0.02412619  0.21416458  0.1188198   0.38227263
 -0.04517859  0.1417562  -0.31381208  0.2106447   0.00822109 -0.13669954
  0.4710996   0.17543544  0.12046697 -0.02408728  0.09891791 -0.08062794
  0.009671    0.2026222  -0.04359178 -0.31278443  0.01

In [79]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors_own.most_similar('universität', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

('hochschule', 0.8408931493759155)
('fakultät', 0.7962464690208435)
('universitäten', 0.7852887511253357)
('ehrendoktor', 0.7707276344299316)
('university', 0.7700812220573425)
('volluniversität', 0.7546650767326355)
('hochschulen', 0.7529446482658386)
('fachhochschule', 0.7368623614311218)
('sorbonne', 0.7329705953598022)
('kunsthochschule', 0.7321016192436218)


In [80]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors_own.most_similar('rot', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

('blau', 0.9078530669212341)
('grün', 0.8994275331497192)
('gelb', 0.8767167925834656)
('schwarz', 0.8723684549331665)
('stadtfarben', 0.8272213935852051)
('stadtflagge', 0.8210878372192383)
('landesfarben', 0.811372697353363)
('silbern', 0.7903183698654175)
('weiß', 0.7887179851531982)
('gestreift', 0.7856178879737854)


In [81]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors_own.most_similar('brüssel', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

('genf', 0.8063685297966003)
('amsterdam', 0.7932760119438171)
('paris', 0.7852495312690735)
('madrid', 0.7727000713348389)
('lüttich', 0.7647874355316162)
('straßburg', 0.7594782710075378)
('charleroi', 0.7566460967063904)
('lille', 0.7544008493423462)
('toulouse', 0.7536197900772095)
('stockholm', 0.7521782517433167)


In [82]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors_own.most_similar('deutschland', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

('österreich', 0.8338862061500549)
('bundesrepublik', 0.8042684197425842)
('schweiz', 0.7832845449447632)
('ostdeutschland', 0.7801434993743896)
('westdeutschland', 0.7517196536064148)
('belgien', 0.725387692451477)
('europa', 0.7193728089332581)
('frankreich', 0.7093033194541931)
('lettland', 0.70259028673172)
('verkehrsclub', 0.698092520236969)


In [83]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors_own.most_similar('komisch', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

('langweilig', 0.8917211294174194)
('sympathisch', 0.8823737502098083)
('witzig', 0.8816604614257812)
('banal', 0.8811015486717224)
('wunderbar', 0.8784763813018799)
('wahrhaftig', 0.8775321245193481)
('hassen', 0.8730990290641785)
('anschauen', 0.8728605508804321)
('tragisch', 0.8708015084266663)
('großartig', 0.86897873878479)


In [84]:
# Compute analogous words
# E.g. x is to queen as man is to king => x = v('queen') + v('man') - v('king')
analogous_words = word_vectors_own.most_similar(negative=['mann'], positive=['könig','frau'])
print('\n'.join([str(tup) for tup in analogous_words]))

('krönung', 0.6899225115776062)
('königin', 0.6776207685470581)
('gemahlin', 0.671101450920105)
('ottos', 0.6473620533943176)
('regentin', 0.6402062773704529)
('theresia', 0.6371099352836609)
('prinzessin', 0.6355440616607666)
('thronfolger', 0.6326956748962402)
('sigismund', 0.6317800879478455)
('isabella', 0.630694568157196)


In [85]:
# Compute analogous words
# E.g. x is to queen as man is to king => x = v('queen') + v('man') - v('king')
analogous_words = word_vectors_own.most_similar(negative=['könig'], positive=['königin','mann'])
print('\n'.join([str(tup) for tup in analogous_words]))

('großmutter', 0.6486426591873169)
('zuviel', 0.6462733149528503)
('frau', 0.6370766162872314)
('ellen', 0.6279744505882263)
('wußte', 0.6237425208091736)
('elizabeth', 0.6159384250640869)
('schauspielerin', 0.6155344247817993)
('anne', 0.6154522895812988)
('alice', 0.6134677529335022)
('tante', 0.6102522611618042)


In [86]:
# Compute analogous words
# E.g. x is to queen as man is to king => x = v('queen') + v('man') - v('king')
analogous_words = word_vectors_own.most_similar(negative=['mutter'], positive=['vater','tochter'])
print('\n'.join([str(tup) for tup in analogous_words]))

('sohn', 0.8599680662155151)
('bruder', 0.7702828049659729)
('großvater', 0.7331442832946777)
('cousin', 0.7274137139320374)
('halbbruder', 0.7147159576416016)
('ehefrau', 0.7091336846351624)
('schwiegersohn', 0.7005618810653687)
('erbin', 0.6972846388816833)
('enkel', 0.691847026348114)
('neffe', 0.6910774111747742)


In [87]:
# Compute analogous words
# E.g. x is to queen as man is to king => x = v('queen') + v('man') - v('king')
analogous_words = word_vectors_own.most_similar(negative=['mutter'], positive=['vater','mädchen'])
print('\n'.join([str(tup) for tup in analogous_words]))

('werthers', 0.6393195986747742)
('jungen', 0.6264751553535461)
('jugendliche', 0.595009446144104)
('lehrer', 0.5943997502326965)
('hauslehrern', 0.5909024477005005)
('hund', 0.5899500250816345)
('prostituierte', 0.5867388844490051)
('musizieren', 0.5775508284568787)
('kleinkinder', 0.5749905109405518)
('engel', 0.5749112367630005)


In [88]:
# Compute analogous words
# x is to daughter as people is to person (plural + daughter)
analogous_words = word_vectors_own.most_similar(negative=['person'], positive=['menschen','tochter'])
print('\n'.join([str(tup) for tup in analogous_words]))

('heirateten', 0.5747276544570923)
('geboren', 0.5573669672012329)
('mutter', 0.5540222525596619)
('geborene', 0.5523591637611389)
('kinder', 0.5452825427055359)
('witwe', 0.5435317158699036)
('heiratete', 0.5411205887794495)
('unehelich', 0.5395071506500244)
('schwester', 0.5393198728561401)
('großmutter', 0.5384270548820496)


In [90]:
analogous_words = word_vectors_own.most_similar(negative=['weich'], positive=['hart','weicher'])
print('\n'.join([str(tup) for tup in analogous_words]))

('kalte', 0.7127009630203247)
('milder', 0.6817352175712585)
('polarluft', 0.6750123500823975)
('warmer', 0.6746388077735901)
('bescheidener', 0.6650974750518799)
('katabatischer', 0.6614716649055481)
('südschweiz', 0.6606355309486389)
('zäh', 0.6599506139755249)
('unangenehm', 0.6589765548706055)
('humides', 0.6531760096549988)


### Georgian examples

In [ ]:
# See the word vector for a particular word -- 'world'
vector = word_vectors_own['სამყარო']
print(vector)

In [ ]:
# See which words are closest to a given word in the vector space = 'world'
similar_words = word_vectors_own.most_similar('არსება', topn=30)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# blue
similar_words = word_vectors_own.most_similar('ლურჯი', topn=30)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# france
similar_words = word_vectors_own.most_similar('საფრანგეთი', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# x is to king as woman is to man
analogous_words = word_vectors_own.most_similar(negative=['კაცი'], positive=['მეფე','ქალი'])
print('\n'.join([str(tup) for tup in analogous_words]))

In [ ]:
# x is to king as woman is to man
# x = small + biggest - big
# 𝑥−𝑣𝑒𝑐𝑡𝑜𝑟(′𝑠𝑚𝑎𝑙𝑙′)=𝑣𝑒𝑐𝑡𝑜𝑟(′𝑏𝑖𝑔𝑔𝑒𝑠𝑡′)−𝑣𝑒𝑐𝑡𝑜𝑟(′𝑏𝑖𝑔′)
# ∴ 𝑥=𝑣𝑒𝑐𝑡𝑜𝑟(′𝑠𝑚𝑎𝑙𝑙′)+𝑣𝑒𝑐𝑡𝑜𝑟(′𝑏𝑖𝑔𝑔𝑒𝑠𝑡′)−𝑣𝑒𝑐𝑡𝑜𝑟(′𝑏𝑖𝑔′)
# ∴ 𝑥=𝑣𝑒𝑐𝑡𝑜𝑟(′փոքր′)+𝑣𝑒𝑐𝑡𝑜𝑟(′ամենամեծը′)−𝑣𝑒𝑐𝑡𝑜𝑟(′մեծ′)
analogous_words = word_vectors_own.most_similar(negative=['პირი'], positive=['ხალხი','ქალიშვილი'])
print('\n'.join([str(tup) for tup in analogous_words]))

In [ ]:
# try your own examples... (also -- morphology?)
# # x is to daughter as people is to person (plural + daughter)
# analogous_words = word_vectors.most_similar(negative=['person'], positive=['people','daughter'])

### Armenian examples

In [ ]:
# See the word vector for a particular word -- 'world'
vector = word_vectors_own['աշխարհը']
print(vector)

In [ ]:
# See which words are closest to a given word in the vector space = 'world'
similar_words = word_vectors_own.most_similar('աշխարհը', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# blue
similar_words = word_vectors_own.most_similar('կապույտ', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# france
similar_words = word_vectors_own.most_similar('ֆրանսիա', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# x is to king as woman is to man
analogous_words = word_vectors_own.most_similar(negative=['մարդ'], positive=['թագավոր','կին'])
print('\n'.join([str(tup) for tup in analogous_words]))

In [ ]:
# x is to king as woman is to man
# x = small + biggest - big
# 𝑥−𝑣𝑒𝑐𝑡𝑜𝑟(′𝑠𝑚𝑎𝑙𝑙′)=𝑣𝑒𝑐𝑡𝑜𝑟(′𝑏𝑖𝑔𝑔𝑒𝑠𝑡′)−𝑣𝑒𝑐𝑡𝑜𝑟(′𝑏𝑖𝑔′)
# ∴ 𝑥=𝑣𝑒𝑐𝑡𝑜𝑟(′𝑠𝑚𝑎𝑙𝑙′)+𝑣𝑒𝑐𝑡𝑜𝑟(′𝑏𝑖𝑔𝑔𝑒𝑠𝑡′)−𝑣𝑒𝑐𝑡𝑜𝑟(′𝑏𝑖𝑔′)
# ∴ 𝑥=𝑣𝑒𝑐𝑡𝑜𝑟(′փոքր′)+𝑣𝑒𝑐𝑡𝑜𝑟(′ամենամեծը′)−𝑣𝑒𝑐𝑡𝑜𝑟(′մեծ′)
analogous_words = word_vectors_own.most_similar(negative=['մեծ'], positive=['ամենամեծը','փոքր'])
print('\n'.join([str(tup) for tup in analogous_words]))

In [ ]:
# try your own examples... (also -- morphology?)
# # x is to daughter as people is to person (plural + daughter)
# analogous_words = word_vectors.most_similar(negative=['person'], positive=['people','daughter'])